# Advanced Certification in AIML
## A Program by IIIT-H and TalentSprint

## Learning Objective

At the end of the experiment, you will be able to:

*  understand clustering
*  find groups or clusters using K-means algorithm

In [ ]:
#@title Experiment Walkthrough Video
from IPython.display import HTML

HTML("""<video width="854" height="480" controls>
  <source src="https://cdn.talentsprint.com/aiml/AIML_BATCH_HYD_7/3March/kmeans.mp4" type="video/mp4">
</video>
""")

## Dataset

### Description

The dataset choosen for this experiment is a hand-made dataset. We have generated the data programmatically by following Random / Gaussian distribution. The dataset includes 3000 instances where each instance is made up of 2 features.

## AIML Technique

### Clustering

Given an unfamiliar dataset, a clustering algorithm helps to discover groups of objects where the average distances between the members of each cluster are closer than to members in other clusters.

Clustering is used to find structure in unlabeled data. It’s the most common form of unsupervised learning.

Several Algorithms have been proposed for clustering. However, when compared to other clustering algorithms the K-means algorithm is more effective. This is because K-means allows us to pre-cluster the database which in turn produces good results.

Additionally, K-means clustering is the simplest to implement and to run. All we need to do is choose the number of clusters(K) and run it number of times.

We use K-means algorithm for clustering in this experiment.

### K-means Algorithm

K-means clustering is the process of partitioning a set of data objects into subsets. Each subset is a cluster such that the similarity within the cluster is greater and the similarity between the other clusters is less.

K-means algorithm iteratively performs the following steps:

1. Determine the value “K”, the value “K” represents the number of clusters.
2. Select "K" points at random as cluster centers.
3. Assign data points to their closest cluster center according to the Euclidean distance function.
4. Calculate the centroid or mean of all the data points in each cluster.
5. Repeat steps 3 and 4 until the same points are assigned to each cluster and compute till convergence.


### Setup Steps:

In [ ]:
#@title Please enter your registration id to start: { run: "auto", display-mode: "form" }
Id = "" #@param {type:"string"}

In [ ]:
#@title Please enter your password (normally your phone number) to continue: { run: "auto", display-mode: "form" }
password = "" #@param {type:"string"}

In [ ]:
#@title Run this cell to complete the setup for this Notebook
from IPython import get_ipython
import re
ipython = get_ipython()

notebook= "U2W7_24_Kmeans_Clustering_A" #name of the notebook

def setup():
#  ipython.magic("sx pip3 install torch")
    from IPython.display import HTML, display
    ipython.magic("sx wget https://cdn.iiith.talentsprint.com/aiml/Experiment_related_data/DS_CLARA_STD.csv")
    display(HTML('<script src="https://dashboard.talentsprint.com/aiml/record_ip.html?traineeId={0}&recordId={1}"></script>'.format(getId(),submission_id)))
    print("Setup completed successfully")
    return

def submit_notebook():
    ipython.magic("notebook -e "+ notebook + ".ipynb")

    import requests, json, base64, datetime

    url = "https://dashboard.talentsprint.com/xp/app/save_notebook_attempts"
    if not submission_id:
      data = {"id" : getId(), "notebook" : notebook, "mobile" : getPassword()}
      r = requests.post(url, data = data)
      r = json.loads(r.text)

      if r["status"] == "Success":
          return r["record_id"]
      elif "err" in r:
        print(r["err"])
        return None
      else:
        print ("Something is wrong, the notebook will not be submitted for grading")
        return None

    elif getAnswer() and getComplexity() and getAdditional() and getConcepts() and getWalkthrough() and getComments() and getMentorSupport():
      f = open(notebook + ".ipynb", "rb")
      file_hash = base64.b64encode(f.read())

      data = {"complexity" : Complexity, "additional" :Additional,
              "concepts" : Concepts, "record_id" : submission_id,
              "answer" : Answer, "id" : Id, "file_hash" : file_hash,
              "notebook" : notebook, "feedback_walkthrough":Walkthrough ,
              "feedback_experiments_input" : Comments,
              "feedback_inclass_mentor": Mentor_support}

      r = requests.post(url, data = data)
      r = json.loads(r.text)
      if "err" in r:
        print(r["err"])
        return None
      else:
        print("Your submission is successful.")
        print("Ref Id:", submission_id)
        print("Date of submission: ", r["date"])
        print("Time of submission: ", r["time"])
        print("View your submissions: https://learn-iiith.talentsprint.com/notebook_submissions")
        #print("For any queries/discrepancies, please connect with mentors through the chat icon in LMS dashboard.")
        return submission_id
    else: submission_id


def getAdditional():
  try:
    if not Additional:
      raise NameError
    else:
      return Additional
  except NameError:
    print ("Please answer Additional Question")
    return None

def getComplexity():
  try:
    if not Complexity:
      raise NameError
    else:
      return Complexity
  except NameError:
    print ("Please answer Complexity Question")
    return None

def getConcepts():
  try:
    if not Concepts:
      raise NameError
    else:
      return Concepts
  except NameError:
    print ("Please answer Concepts Question")
    return None


def getWalkthrough():
  try:
    if not Walkthrough:
      raise NameError
    else:
      return Walkthrough
  except NameError:
    print ("Please answer Walkthrough Question")
    return None

def getComments():
  try:
    if not Comments:
      raise NameError
    else:
      return Comments
  except NameError:
    print ("Please answer Comments Question")
    return None


def getMentorSupport():
  try:
    if not Mentor_support:
      raise NameError
    else:
      return Mentor_support
  except NameError:
    print ("Please answer Mentor support Question")
    return None

def getAnswer():
  try:
    if not Answer:
      raise NameError
    else:
      return Answer
  except NameError:
    print ("Please answer Question")
    return None


def getId():
  try:
    return Id if Id else None
  except NameError:
    return None

def getPassword():
  try:
    return password if password else None
  except NameError:
    return None

submission_id = None
### Setup
if getPassword() and getId():
  submission_id = submit_notebook()
  if submission_id:
    setup()
else:
  print ("Please complete Id and Password cells before running setup")



Let us learn the kmeans algorithm step by step.

## Importing required packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
from copy import deepcopy
import warnings
warnings.filterwarnings("ignore")

## Load the dataset

In [ ]:
# Setting up the file. The data below is a simple Gaussian distribution of data.
dataset = "DS_CLARA_STD.csv"

In [ ]:
data = pd.read_csv(dataset)
print(data.head())

## Get the data values and plot them

In [ ]:
# Storing the values in seperate variables
f1 = data['V1'].values
f2 = data['V2'].values

# Combining f1 and f2 into a single array
X = # YOUR CODE HERE : From the two 1-dim Gaussian variables 'f1' and 'f2', create a 2-d gaussian variable 'X' (Hint: By zipping them together)

## Visualize the data values

In [ ]:
# YOUR CODE HERE : Scatter plot 'f1' vs 'f2'

## Lets intialize three cluster centers randomly to start with the k-means algorithm.

Note: Refer to [np.random.randint](https://numpy.org/doc/stable/reference/random/generated/numpy.random.randint.html)

In [ ]:
# Number of clusters
k = 3
C = # YOUR CODE HERE : Generate random centriods (Hint : Generate x and y random coordinates)

## Lets visualize the data when the algorithm gets intialized along with the Centroids

In [ ]:
# YOUR CODE HERE : Scatter plot 'f1' vs 'f2' and Centriods ('C')

## Apply  K-means algorithm

Now lets learn from above together and see how exactly k-means algorithm works.

Now lets write a function to calculate Euclidean Distance between two array of points 'a' and 'b' or centroid

**Note:** Refer to the following link to understand how [linalg.norm](https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html) works

In [ ]:
def dist(a, b, ax=1):
    return # YOUR CODE HERE : Calculate euclidean distance between a and b
           # NOTE: The input 'a' can either be a point or an array of points (of same length as 'b'); 'b' is an array of points.
           # HINT : You can use 'linalg.norm' function from numpy package

In [ ]:
# To store the value of centroids. Initialize with zero
C_old = np.zeros(C.shape)

# Initialize clusters of x[i] with zeros
clusters = np.zeros(len(X))

# Error func - Distance between new centroids and old centroids
error = dist(C, C_old, None)

# Loop will run till the error becomes zero
fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(111)
plt.ion()       # Plot keeps getting updated in loop, So turning on plot in interactive mode
while error >= 0.1:

    colors = ['r', 'g', 'b', 'y', 'c', 'm']

    # Assigning each sample/data point to its closest cluster

    for i in range(len(X)):
        distances = # YOUR CODE HERE : calculate distance of 'X[i]' from each of the 3 centroid points contained within 'C'.
                   # HINT : Use 'dist' function defined above
        cluster = # YOUR CODE HERE : Out of all the distances calculated above, find the centroid with the least distance
        clusters[i] = cluster    # The cluster center found above is now the new cluster assignment for the point X[i]

    # Storing the old centroid values
    C_old = deepcopy(C)   # we use deep copy here because we don't want the changes in array C to be reflecting in C_old (because assignment is by reference)
    # Finding the new centroids by taking the average value

    for i in range(k):     # We iterate through each of the 3 clusters.
        points = [X[j] for j in range(len(X)) if clusters[j] == i]  # We pull out all the points that were assigned to the new cluster above (in 'clusters' variable)
        C[i] = # YOUR CODE HERE : Using the above generated 'points', recompute the new centroid, by taking the mean.

    error = # YOUR CODE HERE : Calculate the error between the new cluster centres('C') and the old cluster centres ('C_old').
            # Hint : Use 'dist' function defined above
    ax.clear()
    for i in range(k):
        points = np.array([X[j] for j in range(len(X)) if clusters[j] == i])
        plt.scatter(points[:, 0], points[:, 1], s=7, c=colors[i])
        plt.scatter(C[:, 0], C[:, 1], marker='*', s=200, c='w')
    print(f"Error: {error}\n")

## Apply K-means from sklearn
**Note:** Refer to the following [link](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html)

In [ ]:
from sklearn.cluster import KMeans

# YOUR CODE HERE: To create an instance for 'KMeans' and select the number of clusters to generate
# YOUR CODE HERE: To fit and predict the data using k-means

In [ ]:
# YOUR CODE HERE: To print the cluster centroids

## Find the number of clusters using Elbow Method

To determine the optimal number of clusters, we have to select the value of k at the “elbow” ie.., the point after which the distortion/inertia start decreasing in a linear fashion.

In [ ]:
scores = []

for i in range(1, 10):
  kmeans = KMeans(n_clusters = i)
  # YOUR CODE HERE: To fit the data using KMeans
  scores.append(kmeans.inertia_)

plt.plot(range(1, 10), scores, 'bx-')
plt.title('Finding the number of clusters')
plt.xlabel('Clusters')
plt.ylabel('Scores')
plt.show()

From the above Elbow Method we can choose the number of clusters as 3

### Please answer the questions below to complete the experiment:




In [ ]:
#@title State True or False: In this experiment, after initializing the random centroids, the first step inolves, computing the mean? { run: "auto", form-width: "500px", display-mode: "form" }
Answer = "" #@param ["","TRUE","FALSE"]


In [ ]:
#@title How was the experiment? { run: "auto", form-width: "500px", display-mode: "form" }
Complexity = "" #@param ["","Too Simple, I am wasting time", "Good, But Not Challenging for me", "Good and Challenging for me", "Was Tough, but I did it", "Too Difficult for me"]


In [ ]:
#@title If it was too easy, what more would you have liked to be added? If it was very difficult, what would you have liked to have been removed? { run: "auto", display-mode: "form" }
Additional = "" #@param {type:"string"}


In [ ]:
#@title Can you identify the concepts from the lecture which this experiment covered? { run: "auto", vertical-output: true, display-mode: "form" }
Concepts = "" #@param ["","Yes", "No"]


In [ ]:
#@title  Experiment walkthrough video? { run: "auto", vertical-output: true, display-mode: "form" }
Walkthrough = "" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [ ]:
#@title  Text and image description/explanation and code comments within the experiment: { run: "auto", vertical-output: true, display-mode: "form" }
Comments = "" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [ ]:
#@title Mentor Support: { run: "auto", vertical-output: true, display-mode: "form" }
Mentor_support = "" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [ ]:
#@title Run this cell to submit your notebook for grading { vertical-output: true }
try:
  if submission_id:
      return_id = submit_notebook()
      if return_id : submission_id = return_id
  else:
      print("Please complete the setup first.")
except NameError:
  print ("Please complete the setup first.")